In [259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

from sklearn.model_selection import cross_val_score
from collections import Counter as cnt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score as accuracy
from sklearn.model_selection import train_test_split as ttsplit
from catboost import CatBoostClassifier as Cat
from catboost import Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [157]:
data = pd.read_csv('titanic/train.csv')
len(data)

891

## Feature Engineering

In [283]:
df = data.copy()

In [284]:
df = df.drop('PassengerId', 1)
df = df.drop('Name', 1)
df['Embarked'].fillna('S', inplace=True)
df['Age'].fillna(np.mean(df['Age']), inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [285]:
cnt = Counter(df['Cabin'])
len(cnt.most_common()), cnt.most_common()

(148,
 [(nan, 687),
  ('G6', 4),
  ('C23 C25 C27', 4),
  ('B96 B98', 4),
  ('F33', 3),
  ('E101', 3),
  ('F2', 3),
  ('D', 3),
  ('C22 C26', 3),
  ('C123', 2),
  ('D33', 2),
  ('C52', 2),
  ('B28', 2),
  ('C83', 2),
  ('F G73', 2),
  ('D26', 2),
  ('B58 B60', 2),
  ('C2', 2),
  ('E33', 2),
  ('F4', 2),
  ('D36', 2),
  ('C93', 2),
  ('C78', 2),
  ('D35', 2),
  ('B77', 2),
  ('E67', 2),
  ('C125', 2),
  ('B49', 2),
  ('C65', 2),
  ('B57 B59 B63 B66', 2),
  ('B18', 2),
  ('C124', 2),
  ('B35', 2),
  ('E44', 2),
  ('C92', 2),
  ('D20', 2),
  ('E25', 2),
  ('B22', 2),
  ('C68', 2),
  ('C126', 2),
  ('B51 B53 B55', 2),
  ('B5', 2),
  ('B20', 2),
  ('E24', 2),
  ('E8', 2),
  ('E121', 2),
  ('D17', 2),
  ('C85', 1),
  ('E46', 1),
  ('C103', 1),
  ('D56', 1),
  ('A6', 1),
  ('B78', 1),
  ('B30', 1),
  ('E31', 1),
  ('A5', 1),
  ('D10 D12', 1),
  ('C110', 1),
  ('F E69', 1),
  ('D47', 1),
  ('B86', 1),
  ('B19', 1),
  ('A7', 1),
  ('C49', 1),
  ('A32', 1),
  ('B4', 1),
  ('B80', 1),
  ('A31', 1)

In [286]:
# df['Cabin'].fillna('C0', inplace=True)

### Cabin stuff

In [287]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str(big_string).find(substring) != -1:
            return substring
    return np.nan

cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']

df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

In [288]:
def make_deck(row):
    if pd.isnull(row['Deck']):
        row['Deck'] = 'D' + str(row['Pclass'])
    return row

In [289]:
df = df.apply(make_deck, axis=1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,D3
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,D3
3,1,1,female,35.0,1,0,113803,53.1000,C123,S,C
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S,D3


In [290]:
import re

In [291]:
def cab_num(cab):
    if pd.isnull(cab):
        return 50  # about avg
    cab = re.sub(r'[A-Z]', '', cab)
    nums = cab.split()
    if len(nums) == 0:
        return 50
    num = int(cab.split()[0])
    return num

In [292]:
df['Cabin'] = df['Cabin'].apply(cab_num)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,0,3,male,22.0,1,0,A/5 21171,7.2500,50,S,D3
1,1,1,female,38.0,1,0,PC 17599,71.2833,85,C,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,50,S,D3
3,1,1,female,35.0,1,0,113803,53.1000,123,S,C
4,0,3,male,35.0,0,0,373450,8.0500,50,S,D3


### TTSplit

In [294]:
x_train, x_val, y_train, y_val = ttsplit(df.drop('Survived', 1), df['Survived'], test_size=0.2, random_state=42)

### K-fold split

In [255]:
kf = KFold(n_splits=4)
X = df.drop('Survived', 1)
y = df['Survived']

## Models

### CatBoost

In [295]:
x_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin',
       'Embarked', 'Deck'],
      dtype='object')

In [296]:
cat_feats = ['Pclass', 'Sex', 'Embarked', 'Ticket', 'Deck']
#cat_feats = ['Pclass', 'Sex', 'Embarked', 'Ticket', 'Cabin']

In [297]:
cat = Cat(n_estimators=200, max_depth=5, learning_rate=0.09, cat_features=cat_feats)

In [298]:
cat.fit(x_train, y_train)

0:	learn: 0.6459683	total: 2.65ms	remaining: 527ms
1:	learn: 0.6052736	total: 5.85ms	remaining: 579ms
2:	learn: 0.5750648	total: 8.58ms	remaining: 564ms
3:	learn: 0.5386073	total: 11ms	remaining: 537ms
4:	learn: 0.5123543	total: 13.3ms	remaining: 520ms
5:	learn: 0.4915568	total: 15.4ms	remaining: 499ms
6:	learn: 0.4789706	total: 17.5ms	remaining: 484ms
7:	learn: 0.4631279	total: 19.9ms	remaining: 478ms
8:	learn: 0.4503592	total: 22.2ms	remaining: 471ms
9:	learn: 0.4421825	total: 24.4ms	remaining: 463ms
10:	learn: 0.4350315	total: 26.5ms	remaining: 455ms
11:	learn: 0.4286652	total: 28.1ms	remaining: 441ms
12:	learn: 0.4230862	total: 30.4ms	remaining: 438ms
13:	learn: 0.4191839	total: 33.5ms	remaining: 445ms
14:	learn: 0.4157403	total: 39.3ms	remaining: 484ms
15:	learn: 0.4112835	total: 44ms	remaining: 506ms
16:	learn: 0.4069458	total: 48ms	remaining: 517ms
17:	learn: 0.4058167	total: 50.4ms	remaining: 510ms
18:	learn: 0.4033508	total: 52.2ms	remaining: 497ms
19:	learn: 0.4009224	total: 

In [299]:
accuracy(cat.predict(x_val), y_val), accuracy(cat.predict(x_train), y_train)

(0.8156424581005587, 0.9171348314606742)

In [300]:
imp_feats = np.concatenate((cat.get_feature_importance().reshape(-1, 1), np.array(x_train.columns).reshape(-1, 1)), 1).tolist()
imp_feats.sort()
imp_feats[::-1]

[[24.00415252299027, 'Sex'],
 [18.035286693402647, 'Age'],
 [13.320432523911808, 'Fare'],
 [12.75927156076402, 'Deck'],
 [8.907847966885736, 'Pclass'],
 [8.698832264238739, 'Embarked'],
 [5.437732990421665, 'Ticket'],
 [4.054926151726636, 'Cabin'],
 [2.8334781602269046, 'SibSp'],
 [1.948039165431598, 'Parch']]

In [301]:
cross_val_score(cat, df.drop('Survived', 1), df['Survived'], cv=5)

0:	learn: 0.6473821	total: 2.52ms	remaining: 502ms
1:	learn: 0.6020827	total: 5.62ms	remaining: 556ms
2:	learn: 0.5705019	total: 8.32ms	remaining: 547ms
3:	learn: 0.5434123	total: 10.7ms	remaining: 523ms
4:	learn: 0.5179412	total: 13ms	remaining: 507ms
5:	learn: 0.4952143	total: 15.5ms	remaining: 502ms
6:	learn: 0.4773368	total: 17.9ms	remaining: 494ms
7:	learn: 0.4662725	total: 19.1ms	remaining: 458ms
8:	learn: 0.4562841	total: 20.9ms	remaining: 444ms
9:	learn: 0.4507666	total: 22.6ms	remaining: 429ms
10:	learn: 0.4420680	total: 24.8ms	remaining: 426ms
11:	learn: 0.4354436	total: 26.7ms	remaining: 418ms
12:	learn: 0.4277898	total: 28.8ms	remaining: 414ms
13:	learn: 0.4225770	total: 30.7ms	remaining: 408ms
14:	learn: 0.4180824	total: 32.6ms	remaining: 402ms
15:	learn: 0.4124022	total: 34.8ms	remaining: 401ms
16:	learn: 0.4087482	total: 37.2ms	remaining: 400ms
17:	learn: 0.4048873	total: 39.2ms	remaining: 397ms
18:	learn: 0.4029800	total: 42.3ms	remaining: 403ms
19:	learn: 0.3984669	tot

4:	learn: 0.5042445	total: 8.52ms	remaining: 332ms
5:	learn: 0.4860258	total: 10.1ms	remaining: 326ms
6:	learn: 0.4775244	total: 11.2ms	remaining: 307ms
7:	learn: 0.4645323	total: 12.1ms	remaining: 290ms
8:	learn: 0.4514766	total: 13.6ms	remaining: 288ms
9:	learn: 0.4395788	total: 15.4ms	remaining: 294ms
10:	learn: 0.4320023	total: 16.9ms	remaining: 290ms
11:	learn: 0.4256523	total: 18.6ms	remaining: 291ms
12:	learn: 0.4171805	total: 20.1ms	remaining: 289ms
13:	learn: 0.4101593	total: 21.5ms	remaining: 286ms
14:	learn: 0.4068348	total: 23.3ms	remaining: 287ms
15:	learn: 0.4033798	total: 24.8ms	remaining: 285ms
16:	learn: 0.3974045	total: 26.5ms	remaining: 286ms
17:	learn: 0.3937215	total: 27.9ms	remaining: 282ms
18:	learn: 0.3891061	total: 29.7ms	remaining: 283ms
19:	learn: 0.3836738	total: 31.1ms	remaining: 280ms
20:	learn: 0.3822120	total: 32.6ms	remaining: 278ms
21:	learn: 0.3816381	total: 33.6ms	remaining: 271ms
22:	learn: 0.3782790	total: 35.3ms	remaining: 272ms
23:	learn: 0.37651

191:	learn: 0.2166350	total: 383ms	remaining: 16ms
192:	learn: 0.2164347	total: 385ms	remaining: 14ms
193:	learn: 0.2160562	total: 388ms	remaining: 12ms
194:	learn: 0.2156636	total: 389ms	remaining: 9.98ms
195:	learn: 0.2153136	total: 391ms	remaining: 7.98ms
196:	learn: 0.2150016	total: 393ms	remaining: 5.98ms
197:	learn: 0.2145588	total: 396ms	remaining: 4ms
198:	learn: 0.2138329	total: 398ms	remaining: 2ms
199:	learn: 0.2137895	total: 400ms	remaining: 0us
0:	learn: 0.6528705	total: 2.15ms	remaining: 427ms
1:	learn: 0.6139562	total: 3.84ms	remaining: 380ms
2:	learn: 0.5822839	total: 5.49ms	remaining: 360ms
3:	learn: 0.5481866	total: 7.51ms	remaining: 368ms
4:	learn: 0.5261866	total: 9.55ms	remaining: 373ms
5:	learn: 0.5092573	total: 11.3ms	remaining: 365ms
6:	learn: 0.4924773	total: 12.8ms	remaining: 353ms
7:	learn: 0.4770074	total: 14.4ms	remaining: 346ms
8:	learn: 0.4660020	total: 16ms	remaining: 339ms
9:	learn: 0.4619126	total: 16.7ms	remaining: 318ms
10:	learn: 0.4555557	total: 18

0:	learn: 0.6540940	total: 1.34ms	remaining: 268ms
1:	learn: 0.6126026	total: 3.43ms	remaining: 339ms
2:	learn: 0.5787885	total: 5.17ms	remaining: 339ms
3:	learn: 0.5421155	total: 6.81ms	remaining: 334ms
4:	learn: 0.5276497	total: 7.85ms	remaining: 306ms
5:	learn: 0.5078329	total: 9.33ms	remaining: 302ms
6:	learn: 0.4913431	total: 10.7ms	remaining: 295ms
7:	learn: 0.4823105	total: 12.1ms	remaining: 291ms
8:	learn: 0.4666834	total: 13.5ms	remaining: 286ms
9:	learn: 0.4540336	total: 14.9ms	remaining: 283ms
10:	learn: 0.4462454	total: 16.2ms	remaining: 279ms
11:	learn: 0.4399050	total: 17.9ms	remaining: 280ms
12:	learn: 0.4341293	total: 19.4ms	remaining: 280ms
13:	learn: 0.4235155	total: 20.9ms	remaining: 278ms
14:	learn: 0.4176074	total: 22.6ms	remaining: 279ms
15:	learn: 0.4131068	total: 24.4ms	remaining: 280ms
16:	learn: 0.4068585	total: 26ms	remaining: 280ms
17:	learn: 0.4042056	total: 27.8ms	remaining: 281ms
18:	learn: 0.4018686	total: 30.8ms	remaining: 293ms
19:	learn: 0.3973891	tot

12:	learn: 0.4536528	total: 26.5ms	remaining: 381ms
13:	learn: 0.4433793	total: 28.9ms	remaining: 384ms
14:	learn: 0.4360214	total: 30.6ms	remaining: 377ms
15:	learn: 0.4314760	total: 32.4ms	remaining: 372ms
16:	learn: 0.4305588	total: 33.2ms	remaining: 358ms
17:	learn: 0.4272936	total: 34.9ms	remaining: 352ms
18:	learn: 0.4218692	total: 36.3ms	remaining: 346ms
19:	learn: 0.4180634	total: 38ms	remaining: 342ms
20:	learn: 0.4154710	total: 39.6ms	remaining: 337ms
21:	learn: 0.4134399	total: 40.9ms	remaining: 331ms
22:	learn: 0.4120563	total: 42.3ms	remaining: 326ms
23:	learn: 0.4075503	total: 44ms	remaining: 323ms
24:	learn: 0.4048344	total: 46ms	remaining: 322ms
25:	learn: 0.4042206	total: 47.1ms	remaining: 315ms
26:	learn: 0.3998152	total: 49.3ms	remaining: 316ms
27:	learn: 0.3972977	total: 51.6ms	remaining: 317ms
28:	learn: 0.3947674	total: 53.6ms	remaining: 316ms
29:	learn: 0.3916306	total: 56ms	remaining: 317ms
30:	learn: 0.3895313	total: 58ms	remaining: 316ms
31:	learn: 0.3869825	t

array([0.81005587, 0.80898876, 0.85955056, 0.82022472, 0.85393258])

In [256]:
res = []
for train_index, test_index in kf.split(X):
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]
    cat.fit(x_train, y_train)
    res.append(accuracy(cat.predict(x_val), y_val))
print(np.mean(res))

KeyError: "None of [Int64Index([223, 224, 225, 226, 227, 228, 229, 230, 231, 232,\n            ...\n            881, 882, 883, 884, 885, 886, 887, 888, 889, 890],\n           dtype='int64', length=668)] are in the [columns]"

### CatBoost GridSearch

In [249]:
parameters = {'n_estimators':[50, 100, 200, 300], 'max_depth':[4, 5, 6],
              'learning_rate':[0.08, 0.09, 0.1, 0.125, 0.15]}
cat = Cat(cat_features=cat_feats)

In [250]:
clf = GridSearchCV(cat, parameters)

In [251]:
clf.fit(x_train, y_train)

0:	learn: 0.6651252	total: 982us	remaining: 48.1ms
1:	learn: 0.6389527	total: 2.16ms	remaining: 51.8ms
2:	learn: 0.6157770	total: 2.83ms	remaining: 44.3ms
3:	learn: 0.5959551	total: 3.48ms	remaining: 40.1ms
4:	learn: 0.5794165	total: 4.32ms	remaining: 38.9ms
5:	learn: 0.5653538	total: 5ms	remaining: 36.7ms
6:	learn: 0.5507986	total: 5.92ms	remaining: 36.3ms
7:	learn: 0.5379636	total: 6.78ms	remaining: 35.6ms
8:	learn: 0.5263504	total: 7.61ms	remaining: 34.7ms
9:	learn: 0.5153380	total: 8.35ms	remaining: 33.4ms
10:	learn: 0.5070214	total: 9.13ms	remaining: 32.4ms
11:	learn: 0.4985471	total: 9.92ms	remaining: 31.4ms
12:	learn: 0.4910501	total: 10.8ms	remaining: 30.6ms
13:	learn: 0.4831738	total: 11.6ms	remaining: 29.8ms
14:	learn: 0.4786042	total: 12.5ms	remaining: 29.1ms
15:	learn: 0.4728978	total: 13.3ms	remaining: 28.2ms
16:	learn: 0.4677417	total: 14.1ms	remaining: 27.3ms
17:	learn: 0.4643267	total: 14.9ms	remaining: 26.4ms
18:	learn: 0.4616862	total: 15.4ms	remaining: 25.1ms
19:	lea

22:	learn: 0.4392608	total: 21.1ms	remaining: 24.8ms
23:	learn: 0.4357209	total: 22.1ms	remaining: 23.9ms
24:	learn: 0.4341205	total: 23ms	remaining: 23ms
25:	learn: 0.4314268	total: 24.1ms	remaining: 22.2ms
26:	learn: 0.4299626	total: 25ms	remaining: 21.3ms
27:	learn: 0.4284865	total: 25.9ms	remaining: 20.4ms
28:	learn: 0.4254277	total: 26.9ms	remaining: 19.4ms
29:	learn: 0.4228242	total: 27.7ms	remaining: 18.4ms
30:	learn: 0.4223619	total: 28.1ms	remaining: 17.2ms
31:	learn: 0.4204012	total: 28.9ms	remaining: 16.2ms
32:	learn: 0.4179863	total: 29.7ms	remaining: 15.3ms
33:	learn: 0.4153736	total: 30.5ms	remaining: 14.3ms
34:	learn: 0.4138526	total: 31.3ms	remaining: 13.4ms
35:	learn: 0.4116971	total: 32.3ms	remaining: 12.6ms
36:	learn: 0.4093166	total: 33.2ms	remaining: 11.7ms
37:	learn: 0.4072363	total: 34ms	remaining: 10.8ms
38:	learn: 0.4071807	total: 34.6ms	remaining: 9.76ms
39:	learn: 0.4065847	total: 35.4ms	remaining: 8.84ms
40:	learn: 0.4047788	total: 36.2ms	remaining: 7.94ms
4

83:	learn: 0.3868441	total: 48.8ms	remaining: 9.29ms
84:	learn: 0.3866308	total: 49.4ms	remaining: 8.72ms
85:	learn: 0.3853378	total: 50.1ms	remaining: 8.15ms
86:	learn: 0.3843796	total: 50.7ms	remaining: 7.57ms
87:	learn: 0.3841165	total: 51.3ms	remaining: 6.99ms
88:	learn: 0.3840883	total: 51.9ms	remaining: 6.41ms
89:	learn: 0.3831697	total: 52.5ms	remaining: 5.83ms
90:	learn: 0.3826295	total: 53.1ms	remaining: 5.25ms
91:	learn: 0.3824004	total: 53.7ms	remaining: 4.67ms
92:	learn: 0.3817399	total: 54.3ms	remaining: 4.08ms
93:	learn: 0.3809433	total: 54.9ms	remaining: 3.5ms
94:	learn: 0.3800652	total: 55.5ms	remaining: 2.92ms
95:	learn: 0.3798042	total: 56.1ms	remaining: 2.34ms
96:	learn: 0.3797077	total: 56.7ms	remaining: 1.75ms
97:	learn: 0.3792385	total: 57.2ms	remaining: 1.17ms
98:	learn: 0.3790025	total: 57.8ms	remaining: 584us
99:	learn: 0.3784570	total: 58.5ms	remaining: 0us
0:	learn: 0.6654761	total: 727us	remaining: 72ms
1:	learn: 0.6399950	total: 1.61ms	remaining: 79.1ms
2:	

57:	learn: 0.4011010	total: 39.1ms	remaining: 28.3ms
58:	learn: 0.4008997	total: 39.7ms	remaining: 27.6ms
59:	learn: 0.3999502	total: 40.3ms	remaining: 26.9ms
60:	learn: 0.3994307	total: 40.9ms	remaining: 26.2ms
61:	learn: 0.3991348	total: 41.7ms	remaining: 25.6ms
62:	learn: 0.3988773	total: 42.4ms	remaining: 24.9ms
63:	learn: 0.3984399	total: 43ms	remaining: 24.2ms
64:	learn: 0.3969152	total: 43.6ms	remaining: 23.5ms
65:	learn: 0.3958121	total: 44.3ms	remaining: 22.8ms
66:	learn: 0.3955190	total: 44.8ms	remaining: 22.1ms
67:	learn: 0.3942917	total: 45.4ms	remaining: 21.4ms
68:	learn: 0.3941249	total: 46.1ms	remaining: 20.7ms
69:	learn: 0.3938209	total: 46.7ms	remaining: 20ms
70:	learn: 0.3928932	total: 47.4ms	remaining: 19.4ms
71:	learn: 0.3920988	total: 48ms	remaining: 18.7ms
72:	learn: 0.3915117	total: 48.6ms	remaining: 18ms
73:	learn: 0.3913007	total: 49.3ms	remaining: 17.3ms
74:	learn: 0.3911581	total: 49.9ms	remaining: 16.6ms
75:	learn: 0.3904165	total: 50.5ms	remaining: 15.9ms
7

0:	learn: 0.6466022	total: 1.67ms	remaining: 333ms
1:	learn: 0.6136674	total: 3.49ms	remaining: 345ms
2:	learn: 0.5786484	total: 4.83ms	remaining: 317ms
3:	learn: 0.5526253	total: 6.12ms	remaining: 300ms
4:	learn: 0.5335030	total: 7.45ms	remaining: 290ms
5:	learn: 0.5165712	total: 8.56ms	remaining: 277ms
6:	learn: 0.5039866	total: 9.99ms	remaining: 275ms
7:	learn: 0.4863788	total: 11.3ms	remaining: 270ms
8:	learn: 0.4803703	total: 12.4ms	remaining: 264ms
9:	learn: 0.4718256	total: 13.8ms	remaining: 262ms
10:	learn: 0.4602895	total: 15.5ms	remaining: 266ms
11:	learn: 0.4521689	total: 16.8ms	remaining: 263ms
12:	learn: 0.4477479	total: 18ms	remaining: 259ms
13:	learn: 0.4417236	total: 19.5ms	remaining: 258ms
14:	learn: 0.4363561	total: 20.5ms	remaining: 253ms
15:	learn: 0.4300023	total: 21.8ms	remaining: 250ms
16:	learn: 0.4248683	total: 23.3ms	remaining: 251ms
17:	learn: 0.4225702	total: 24.6ms	remaining: 249ms
18:	learn: 0.4184995	total: 26.2ms	remaining: 249ms
19:	learn: 0.4126911	tot

...

162:	learn: 0.1686606	total: 341ms	remaining: 286ms
163:	learn: 0.1682623	total: 344ms	remaining: 285ms
164:	learn: 0.1673057	total: 346ms	remaining: 283ms
165:	learn: 0.1670035	total: 348ms	remaining: 281ms
166:	learn: 0.1655621	total: 350ms	remaining: 279ms
167:	learn: 0.1654247	total: 352ms	remaining: 276ms
168:	learn: 0.1645130	total: 354ms	remaining: 274ms
169:	learn: 0.1630268	total: 356ms	remaining: 272ms
170:	learn: 0.1628312	total: 359ms	remaining: 270ms
171:	learn: 0.1619917	total: 361ms	remaining: 268ms
172:	learn: 0.1606469	total: 363ms	remaining: 267ms
173:	learn: 0.1599085	total: 366ms	remaining: 265ms
174:	learn: 0.1594818	total: 368ms	remaining: 263ms
175:	learn: 0.1594010	total: 370ms	remaining: 261ms
176:	learn: 0.1592908	total: 373ms	remaining: 259ms
177:	learn: 0.1577491	total: 375ms	remaining: 257ms
178:	learn: 0.1575472	total: 377ms	remaining: 255ms
179:	learn: 0.1568827	total: 379ms	remaining: 253ms
180:	learn: 0.1563897	total: 381ms	remaining: 251ms
181:	learn: 

31:	learn: 0.3474797	total: 57.7ms	remaining: 484ms
32:	learn: 0.3474797	total: 58.4ms	remaining: 472ms
33:	learn: 0.3454383	total: 60.1ms	remaining: 470ms
34:	learn: 0.3454383	total: 60.7ms	remaining: 459ms
35:	learn: 0.3442803	total: 61.6ms	remaining: 452ms
36:	learn: 0.3432826	total: 62.6ms	remaining: 445ms
37:	learn: 0.3432824	total: 63.1ms	remaining: 435ms
38:	learn: 0.3416060	total: 64.6ms	remaining: 432ms
39:	learn: 0.3408945	total: 66.4ms	remaining: 432ms
40:	learn: 0.3391332	total: 68.4ms	remaining: 432ms
41:	learn: 0.3389405	total: 69.2ms	remaining: 425ms
42:	learn: 0.3370022	total: 70.7ms	remaining: 423ms
43:	learn: 0.3344206	total: 72.6ms	remaining: 423ms
44:	learn: 0.3340755	total: 74.5ms	remaining: 422ms
45:	learn: 0.3313788	total: 76.3ms	remaining: 421ms
46:	learn: 0.3298740	total: 78.2ms	remaining: 421ms
47:	learn: 0.3272108	total: 81.1ms	remaining: 426ms
48:	learn: 0.3244166	total: 83.1ms	remaining: 426ms
49:	learn: 0.3221518	total: 85.1ms	remaining: 425ms
50:	learn: 0

214:	learn: 0.1313862	total: 436ms	remaining: 172ms
215:	learn: 0.1312117	total: 439ms	remaining: 171ms
216:	learn: 0.1304692	total: 441ms	remaining: 169ms
217:	learn: 0.1301677	total: 444ms	remaining: 167ms
218:	learn: 0.1296844	total: 447ms	remaining: 165ms
219:	learn: 0.1288569	total: 449ms	remaining: 163ms
220:	learn: 0.1285088	total: 452ms	remaining: 162ms
221:	learn: 0.1281010	total: 454ms	remaining: 160ms
222:	learn: 0.1280106	total: 457ms	remaining: 158ms
223:	learn: 0.1279439	total: 459ms	remaining: 156ms
224:	learn: 0.1272391	total: 461ms	remaining: 154ms
225:	learn: 0.1267670	total: 464ms	remaining: 152ms
226:	learn: 0.1266992	total: 466ms	remaining: 150ms
227:	learn: 0.1265062	total: 468ms	remaining: 148ms
228:	learn: 0.1264159	total: 470ms	remaining: 146ms
229:	learn: 0.1260616	total: 472ms	remaining: 144ms
230:	learn: 0.1255325	total: 474ms	remaining: 142ms
231:	learn: 0.1251881	total: 477ms	remaining: 140ms
232:	learn: 0.1250700	total: 479ms	remaining: 138ms
233:	learn: 

85:	learn: 0.3211462	total: 157ms	remaining: 208ms
86:	learn: 0.3205776	total: 159ms	remaining: 206ms
87:	learn: 0.3198498	total: 161ms	remaining: 205ms
88:	learn: 0.3193512	total: 163ms	remaining: 203ms
89:	learn: 0.3190335	total: 165ms	remaining: 201ms
90:	learn: 0.3187785	total: 166ms	remaining: 199ms
91:	learn: 0.3174871	total: 168ms	remaining: 197ms
92:	learn: 0.3174209	total: 169ms	remaining: 194ms
93:	learn: 0.3173281	total: 170ms	remaining: 192ms
94:	learn: 0.3168974	total: 172ms	remaining: 191ms
95:	learn: 0.3164807	total: 174ms	remaining: 189ms
96:	learn: 0.3160728	total: 176ms	remaining: 187ms
97:	learn: 0.3157646	total: 178ms	remaining: 185ms
98:	learn: 0.3145750	total: 180ms	remaining: 183ms
99:	learn: 0.3135341	total: 181ms	remaining: 181ms
100:	learn: 0.3115577	total: 183ms	remaining: 180ms
101:	learn: 0.3106863	total: 185ms	remaining: 178ms
102:	learn: 0.3102976	total: 187ms	remaining: 176ms
103:	learn: 0.3099861	total: 189ms	remaining: 175ms
104:	learn: 0.3092639	total

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7ff509c56a60>,
             param_grid={'learning_rate': [0.08, 0.09, 0.1, 0.125, 0.15],
                         'max_depth': [4, 5, 6],
                         'n_estimators': [50, 100, 200, 300]})

In [252]:
grid_results = np.concatenate((np.array(clf.cv_results_['mean_test_score']).reshape(-1, 1),
                          np.array(clf.cv_results_['params']).reshape(-1, 1)), 1).tolist()
grid_results.sort(key=lambda x: x[0])
grid_results[::-1]

[[0.8399192356938837,
  {'learning_rate': 0.09, 'max_depth': 5, 'n_estimators': 200}],
 [0.8384910863784103,
  {'learning_rate': 0.125, 'max_depth': 4, 'n_estimators': 200}],
 [0.837092484979809,
  {'learning_rate': 0.09, 'max_depth': 4, 'n_estimators': 300}],
 [0.8370629370629372,
  {'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 100}],
 [0.8357037328868315,
  {'learning_rate': 0.08, 'max_depth': 5, 'n_estimators': 200}],
 [0.834324830099478,
  {'learning_rate': 0.125, 'max_depth': 5, 'n_estimators': 300}],
 [0.83430513148823,
  {'learning_rate': 0.08, 'max_depth': 6, 'n_estimators': 200}],
 [0.834295282182606,
  {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300}],
 [0.8329262287008765,
  {'learning_rate': 0.125, 'max_depth': 5, 'n_estimators': 200}],
 [0.8329065300896286,
  {'learning_rate': 0.09, 'max_depth': 6, 'n_estimators': 100}],
 [0.8328868314783808,
  {'learning_rate': 0.09, 'max_depth': 5, 'n_estimators': 300}],
 [0.8328769821727569,
  {'learning_rate': 0.12

Best:
[[0.8399192356938837,
  {'learning_rate': 0.09, 'max_depth': 5, 'n_estimators': 200}],
  
  
 [0.8384910863784103,
  {'learning_rate': 0.125, 'max_depth': 4, 'n_estimators': 200}],
  
  
 [0.837092484979809,
  {'learning_rate': 0.09, 'max_depth': 4, 'n_estimators': 300}],

## Submission model

In [302]:
df_test = pd.read_csv('titanic/test.csv')

In [303]:
test_ids = df_test['PassengerId']
df_test = df_test.drop('PassengerId', 1)
df_test = df_test.drop('Name', 1)
df_test['Embarked'].fillna('S', inplace=True)
df_test['Age'].fillna(np.mean(df['Age']), inplace=True)
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,3,female,47.0,1,0,363272,7.0000,NaN,S
2,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,3,male,27.0,0,0,315154,8.6625,NaN,S
4,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [304]:
df_test['Deck'] = df_test['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
df_test = df_test.apply(make_deck, axis=1)
df_test['Cabin'] = df_test['Cabin'].apply(cab_num)

In [305]:
model = Cat(n_estimators=200, max_depth=5, learning_rate=0.09, cat_features=cat_feats)
model.fit(df.drop('Survived', 1), df['Survived'])

0:	learn: 0.6457897	total: 2.44ms	remaining: 485ms
1:	learn: 0.5972164	total: 5.26ms	remaining: 521ms
2:	learn: 0.5661936	total: 7.73ms	remaining: 507ms
3:	learn: 0.5375699	total: 10ms	remaining: 492ms
4:	learn: 0.5140738	total: 12.5ms	remaining: 488ms
5:	learn: 0.4969492	total: 14.3ms	remaining: 461ms
6:	learn: 0.4840235	total: 16.4ms	remaining: 453ms
7:	learn: 0.4662563	total: 18.4ms	remaining: 441ms
8:	learn: 0.4549361	total: 20.5ms	remaining: 434ms
9:	learn: 0.4468155	total: 22.7ms	remaining: 430ms
10:	learn: 0.4379063	total: 25.2ms	remaining: 433ms
11:	learn: 0.4296814	total: 27.8ms	remaining: 435ms
12:	learn: 0.4242976	total: 30.5ms	remaining: 439ms
13:	learn: 0.4190711	total: 32.5ms	remaining: 431ms
14:	learn: 0.4152841	total: 35.3ms	remaining: 435ms
15:	learn: 0.4106361	total: 37.1ms	remaining: 427ms
16:	learn: 0.4055634	total: 39.6ms	remaining: 426ms
17:	learn: 0.3996433	total: 41.5ms	remaining: 420ms
18:	learn: 0.3973696	total: 43.2ms	remaining: 411ms
19:	learn: 0.3936699	tot

In [306]:
predictions = model.predict(df_test)

In [307]:
df_kaggle = pd.DataFrame(test_ids)
df_kaggle['Survived'] = predictions
df_kaggle.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [308]:
df_kaggle.to_csv('submissions/cat-200-5-09_Deck_CabFillna-50.csv', index=False)